In [1]:
import requests
from bofire.data_models.domain.api import Domain
from enum import Enum
import datetime

from pydantic import BaseModel, Field
from bofire.data_models.strategies.api import AnyStrategy, RandomStrategy, SoboStrategy, StepwiseStrategy, Step, AlwaysTrueCondition, NumberOfExperimentsCondition
from typing import Optional
from bofire.data_models.features.api import ContinuousInput

from bofire.data_models.dataframes.api import Experiments, Candidates
from bofire.benchmarks.api import Himmelblau, DTLZ2
import json


class ProposalRequest(BaseModel):
    strategy_data: AnyStrategy
    n_candidates: int = Field(
        default=1, gt=0, description="Number of candidates to generate"
    )
    experiments: Optional[Experiments] = None
    pendings: Optional[Candidates] = None

class StateEnum(str,Enum):
    CREATED = "CREATED"
    CLAIMED = "CLAIMED"
    FAILED = "FAILED"
    FINISHED = "FINISHED"


class Proposal(ProposalRequest):
    id: Optional[int] = None
    candidates: Optional[Candidates] = None
    created_at: datetime.datetime
    last_updated_at: datetime.datetime
    state: StateEnum
    error_message: Optional[str] = None



bench = Himmelblau()
bench2 = DTLZ2(dim=6)
experiments = bench.f(bench.domain.inputs.sample(12), return_complete=True)
experiments2 = bench2.f(bench2.domain.inputs.sample(10), return_complete=True)

strategy_data = StepwiseStrategy(
    domain=bench.domain,
    steps=[
        Step(
            condition=NumberOfExperimentsCondition(
                n_experiments=10
            ),
            strategy_data=RandomStrategy(domain=bench.domain)
        ),
        Step(
            condition=AlwaysTrueCondition(),
            strategy_data=SoboStrategy(domain=bench.domain)
        )
    ]
)


cr = ProposalRequest(
    strategy_data=strategy_data,
    n_candidates=5,
    experiments=None, #Experiments.from_pandas(experiments2, bench2.domain),
    pendings=None
)

cr2 = ProposalRequest(
    strategy_data=SoboStrategy(domain=bench.domain),
    n_candidates=1,
    experiments=None, #Experiments.from_pandas(experiments2, bench2.domain),
    pendings=None
)

cr3 = ProposalRequest(
    strategy_data=SoboStrategy(domain=bench.domain),
    n_candidates=1,
    experiments=Experiments.from_pandas(experiments, bench.domain),
    pendings=None
)


URL = "http://127.0.0.1:8000/"
HEADERS = {'accept': 'application/json', 'Content-Type': 'application/json'}

#response = requests.post(url=f"{URL}/candidates/generate", data=cr.model_dump_json(), headers=HEADERS)

response = requests.post(url=f"{URL}/proposals", json=cr.model_dump(), headers=HEADERS)



/opt/homebrew/Caskroom/miniforge/base/envs/bofire-2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get all proposals

In [2]:
response = requests.get(url=f"{URL}proposals", headers=HEADERS)
print(response.json()[0]["id"])


1


# Use this method to poll

In [3]:
response = requests.get(url=f"{URL}proposals/1/state", headers=HEADERS)
print(response.json())

CREATED


## Use this method to get the candidates

In [4]:
response = requests.get(url=f"{URL}proposals/1/candidates", headers=HEADERS)
Candidates(**response.json()).to_pandas()

,x_1,x_2
0,-1.067388,-1.294144
1,0.453277,4.174079
2,5.976976,-0.614939
3,5.305057,2.977705
4,5.130693,-3.056468


In [25]:
response = requests.post(url=f"{URL}proposals/1/mark_failed", json={"msg": "test"}, headers=HEADERS)
print(response.json())

FAILED


In [74]:
response = requests.get(url=f"{URL}proposals/10/error_message", headers=HEADERS)
print(response.json())

In [75]:
response.content

b'""'

In [31]:
candidates = Candidates.from_pandas(bench.domain.inputs.sample(5), bench.domain)

In [32]:
response = requests.post(url=f"{URL}proposals/1/mark_processed", json=candidates.model_dump(), headers=HEADERS)
print(response.json())

FINISHED


In [82]:
response = requests.get(url=f"{URL}proposals/12/candidates", headers=HEADERS)
Candidates(**response.json()).to_pandas()
#response.content

,x_1,x_2,y_pred,y_sd,y_des
0,-4.046225,-6.0,-38.452175,165.308586,38.452175


In [42]:
response = requests.get(url=f"{URL}proposals/claim", headers=HEADERS)
response.content

b'[5,{"type":"StepwiseStrategy","domain":{"type":"Domain","inputs":{"type":"Inputs","features":[{"type":"ContinuousInput","key":"x_1","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_2","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null}]},"outputs":{"type":"Outputs","features":[{"type":"ContinuousOutput","key":"y","unit":null,"objective":{"type":"MinimizeObjective","w":1.0,"bounds":[0.0,1.0]}}]},"constraints":{"type":"Constraints","constraints":[]}},"seed":null,"steps":[{"type":"Step","strategy_data":{"type":"RandomStrategy","domain":{"type":"Domain","inputs":{"type":"Inputs","features":[{"type":"ContinuousInput","key":"x_1","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_2","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null}]},"outputs":{"type":"Outputs","features":[{"type":"ContinuousOutput","key":"y","un

In [47]:
from pydantic import TypeAdapter

TypeAdapter(AnyStrategy).validate_python(strategy_data.model_dump())

#TypeAdapterAnyStrategy(**strategy_data.model_dump())

StepwiseStrategy(type='StepwiseStrategy', domain=Domain(type='Domain', inputs=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', unit=None, bounds=(-6.0, 6.0), local_relative_bounds=None, stepsize=None), ContinuousInput(type='ContinuousInput', key='x_2', unit=None, bounds=(-6.0, 6.0), local_relative_bounds=None, stepsize=None)]), outputs=Outputs(type='Outputs', features=[ContinuousOutput(type='ContinuousOutput', key='y', unit=None, objective=MinimizeObjective(type='MinimizeObjective', w=1.0, bounds=(0.0, 1.0)))]), constraints=Constraints(type='Constraints', constraints=[])), seed=None, steps=[Step(type='Step', strategy_data=RandomStrategy(type='RandomStrategy', domain=Domain(type='Domain', inputs=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', unit=None, bounds=(-6.0, 6.0), local_relative_bounds=None, stepsize=None), ContinuousInput(type='ContinuousInput', key='x_2', unit=None, bounds=(-6.0, 6.0), local_relative_bounds=

In [50]:
TypeAdapter(AnyStrategy).validate_python(json.loads(response.content)[1])


StepwiseStrategy(type='StepwiseStrategy', domain=Domain(type='Domain', inputs=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', unit=None, bounds=(-6.0, 6.0), local_relative_bounds=None, stepsize=None), ContinuousInput(type='ContinuousInput', key='x_2', unit=None, bounds=(-6.0, 6.0), local_relative_bounds=None, stepsize=None)]), outputs=Outputs(type='Outputs', features=[ContinuousOutput(type='ContinuousOutput', key='y', unit=None, objective=MinimizeObjective(type='MinimizeObjective', w=1.0, bounds=(0.0, 1.0)))]), constraints=Constraints(type='Constraints', constraints=[])), seed=None, steps=[Step(type='Step', strategy_data=RandomStrategy(type='RandomStrategy', domain=Domain(type='Domain', inputs=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', unit=None, bounds=(-6.0, 6.0), local_relative_bounds=None, stepsize=None), ContinuousInput(type='ContinuousInput', key='x_2', unit=None, bounds=(-6.0, 6.0), local_relative_bounds=

In [17]:
response = requests.get(url=f"{URL}proposals/claim", headers=HEADERS)
response.content

b'{"detail":[{"type":"int_parsing","loc":["path","proposal_id"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"claim"}]}'

In [2]:
response = requests.get(url=f"{URL}proposals/", headers=HEADERS)
len(json.loads(response.content))

3

In [76]:
#TypeAdapter(RandomStrategy).validate_python(d["strategy_data"]["steps"][0]["strategy_data"])

TypeAdapter(RandomStrategy).validate_json(json.dumps(d["strategy_data"]["steps"][0]["strategy_data"]))

ValidationError: 1 validation error for RandomStrategy
fallback_sampling_method
  Input should be 'UNIFORM', 'SOBOL' or 'LHS' [type=enum, input_value='SamplingMethodEnum.UNIFORM', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/enum

In [72]:
strategy_data.steps[0].strategy_data.model_dump()["fallback_sampling_method"]

<SamplingMethodEnum.UNIFORM: 'UNIFORM'>

In [8]:
from pydantic import TypeAdapter

# with open("diff.txt", "w") as f:
#     f.write(json.dumps(strategy_data.steps[0].strategy_data.model_dump(), default=str))
#     f.write("\n")
#     f.write(strategy_data.steps[0].strategy_data.model_dump_json())

dump1 = strategy_data.steps[0].strategy_data.model_dump_json()
dump2 = json.dumps(strategy_data.steps[0].strategy_data.model_dump())
TypeAdapter(RandomStrategy).validate_json(dump2)

RandomStrategy(type='RandomStrategy', domain=Domain(type='Domain', inputs=Inputs(type='Inputs', features=[ContinuousInput(type='ContinuousInput', key='x_1', unit=None, bounds=(-6.0, 6.0), local_relative_bounds=None, stepsize=None), ContinuousInput(type='ContinuousInput', key='x_2', unit=None, bounds=(-6.0, 6.0), local_relative_bounds=None, stepsize=None)]), outputs=Outputs(type='Outputs', features=[ContinuousOutput(type='ContinuousOutput', key='y', unit=None, objective=MinimizeObjective(type='MinimizeObjective', w=1.0, bounds=(0.0, 1.0)))]), constraints=Constraints(type='Constraints', constraints=[])), seed=None, fallback_sampling_method=<SamplingMethodEnum.UNIFORM: 'UNIFORM'>, n_burnin=1000, n_thinning=32, num_base_samples=None, max_iters=1000)

In [6]:
dump1

'{"type":"RandomStrategy","domain":{"type":"Domain","inputs":{"type":"Inputs","features":[{"type":"ContinuousInput","key":"x_1","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_2","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null}]},"outputs":{"type":"Outputs","features":[{"type":"ContinuousOutput","key":"y","unit":null,"objective":{"type":"MinimizeObjective","w":1.0,"bounds":[0.0,1.0]}}]},"constraints":{"type":"Constraints","constraints":[]}},"seed":null,"fallback_sampling_method":"UNIFORM","n_burnin":1000,"n_thinning":32,"num_base_samples":null,"max_iters":1000}'

In [67]:
from pydantic import TypeAdapter

strategy_data = TypeAdapter(RandomStrategy).validate_python(d["strategy_data"]["steps"][0]["strategy_data"])


ValidationError: 1 validation error for RandomStrategy
fallback_sampling_method
  Input should be 'UNIFORM', 'SOBOL' or 'LHS' [type=enum, input_value='SamplingMethodEnum.UNIFORM', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/enum

In [64]:
d["strategy_data"]["steps"][0]["strategy_data"]

{'type': 'RandomStrategy',
 'domain': {'type': 'Domain',
  'inputs': {'type': 'Inputs',
   'features': [{'type': 'ContinuousInput',
     'key': 'x_1',
     'unit': None,
     'bounds': [-6.0, 6.0],
     'local_relative_bounds': None,
     'stepsize': None},
    {'type': 'ContinuousInput',
     'key': 'x_2',
     'unit': None,
     'bounds': [-6.0, 6.0],
     'local_relative_bounds': None,
     'stepsize': None}]},
  'outputs': {'type': 'Outputs',
   'features': [{'type': 'ContinuousOutput',
     'key': 'y',
     'unit': None,
     'objective': {'type': 'MinimizeObjective',
      'w': 1.0,
      'bounds': [0.0, 1.0]}}]},
  'constraints': {'type': 'Constraints', 'constraints': []}},
 'seed': None,
 'fallback_sampling_method': 'SamplingMethodEnum.UNIFORM',
 'n_burnin': 1000,
 'n_thinning': 32,
 'num_base_samples': None,
 'max_iters': 1000}

In [58]:
from tinydb import TinyDB

DBPATH = "db.json"
db = TinyDB(DBPATH, default=str)

db.all()[0].keys()

d={**db.all()[0]}

Proposal(**d)

ValidationError: 1169 validation errors for Proposal
strategy_data.function-after[validate_multitask_allowed(), function-after[validate_outlier_detection_specs_for_domain(), function-after[validate_surrogate_specs(), function-after[validate_interpoint_constraints(), function-after[validate_local_search_config(), SoboStrategy]]]]].type
  Input should be 'SoboStrategy' [type=literal_error, input_value='StepwiseStrategy', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
strategy_data.function-after[validate_multitask_allowed(), function-after[validate_outlier_detection_specs_for_domain(), function-after[validate_surrogate_specs(), function-after[validate_interpoint_constraints(), function-after[validate_local_search_config(), SoboStrategy]]]]].steps
  Extra inputs are not permitted [type=extra_forbidden, input_value=[{'type': 'Step', 'strate...n'}, 'transform': None}], input_type=list]
    For further information visit https://errors.pydant

In [56]:
{**db.all()[0]}

{'strategy_data': {'type': 'StepwiseStrategy',
  'domain': {'type': 'Domain',
   'inputs': {'type': 'Inputs',
    'features': [{'type': 'ContinuousInput',
      'key': 'x_1',
      'unit': None,
      'bounds': [-6.0, 6.0],
      'local_relative_bounds': None,
      'stepsize': None},
     {'type': 'ContinuousInput',
      'key': 'x_2',
      'unit': None,
      'bounds': [-6.0, 6.0],
      'local_relative_bounds': None,
      'stepsize': None}]},
   'outputs': {'type': 'Outputs',
    'features': [{'type': 'ContinuousOutput',
      'key': 'y',
      'unit': None,
      'objective': {'type': 'MinimizeObjective',
       'w': 1.0,
       'bounds': [0.0, 1.0]}}]},
   'constraints': {'type': 'Constraints', 'constraints': []}},
  'seed': None,
  'steps': [{'type': 'Step',
    'strategy_data': {'type': 'RandomStrategy',
     'domain': {'type': 'Domain',
      'inputs': {'type': 'Inputs',
       'features': [{'type': 'ContinuousInput',
         'key': 'x_1',
         'unit': None,
         'bo

In [54]:
json.loads(response.content)["doc_id"]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
json.dumps(SoboStrategy(domain=bench.domain).model_dump(), default=str)

'{"type": "SoboStrategy", "domain": {"type": "Domain", "inputs": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0], "local_relative_bounds": null, "stepsize": null}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0], "local_relative_bounds": null, "stepsize": null}]}, "outputs": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "y", "unit": null, "objective": {"type": "MinimizeObjective", "w": 1.0, "bounds": [0, 1]}}]}, "constraints": {"type": "Constraints", "constraints": []}}, "seed": null, "num_restarts": 8, "num_raw_samples": 1024, "maxiter": 2000, "batch_limit": 8, "descriptor_method": "CategoricalMethodEnum.EXHAUSTIVE", "categorical_method": "CategoricalMethodEnum.EXHAUSTIVE", "discrete_method": "CategoricalMethodEnum.EXHAUSTIVE", "surrogate_specs": {"surrogates": [{"hyperconfig": {"type": "SingleTaskGPHyperconfig", "hyperstrategy": "FactorialStrategy", "inputs": {"type": "I

In [9]:
response.content

b'{"rows":[{"inputs":{"x_1":5.250564528500421,"x_2":0.5454267437461127},"outputs":{},"type":"CandidateRow"},{"inputs":{"x_1":5.7480048566119635,"x_2":-5.5182970493448655},"outputs":{},"type":"CandidateRow"},{"inputs":{"x_1":-4.77877335112983,"x_2":1.7390731664552508},"outputs":{},"type":"CandidateRow"},{"inputs":{"x_1":-1.9152471978905758,"x_2":-4.32988876304478},"outputs":{},"type":"CandidateRow"},{"inputs":{"x_1":-3.7801017554643632,"x_2":4.858108150579474},"outputs":{},"type":"CandidateRow"}],"type":"Candidates"}'

In [ ]:
import json
from typing import Optional

import requests
from bofire.benchmarks.api import Himmelblau
from bofire.data_models.dataframes.api import Candidates, Experiments
from bofire.data_models.strategies.api import (
    AlwaysTrueCondition,
    AnyStrategy,
    NumberOfExperimentsCondition,
    RandomStrategy,
    SoboStrategy,
    Step,
    StepwiseStrategy,
)
from pydantic import BaseModel


class CandidateRequest(BaseModel):
    strategy_data: AnyStrategy
    n_candidates: int
    experiments: Optional[Experiments]
    pendings: Optional[Candidates]

# generate experimental data from the Himmelblau benchmark
bench = Himmelblau()
experiments = bench.f(bench.domain.inputs.sample(10), return_complete=True)

# setup the strategy for which candidates should be generated
strategy_data = StepwiseStrategy(
    domain=bench.domain,
    steps=[
        Step(
            condition=NumberOfExperimentsCondition(
                n_experiments=10
            ),
            strategy_data=RandomStrategy(domain=bench.domain)
        ),
        Step(
            condition=AlwaysTrueCondition(),
            strategy_data=SoboStrategy(domain=bench.domain)
        )
    ]
)

# create the payload
payload = CandidateRequest(
    strategy_data=strategy_data,
    n_candidates=1,
    experiments=Experiments.from_pandas(experiments, bench.domain),
    pendings=None
)


URL = "http://127.0.0.1:8000/candidates"
HEADERS = {'accept': 'application/json', 'Content-Type': 'application/json'}

# request candidates
response = requests.post(url=f"{URL}/generate", data=payload.model_dump_json(), headers=HEADERS)

# convert response to a pandas dataframe
df_candidates =Candidates(**json.loads(response.content)).to_pandas()


/opt/homebrew/Caskroom/miniforge/base/envs/bofire-2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
response.content

b'{"rows":[{"inputs":{"x_1":3.218290337622168,"x_2":-0.26658846050092144},"outputs":{},"type":"CandidateRow"}],"type":"Candidates"}'

In [22]:
json.loads(response.content)["detail"][0]["msg"]

'Value error, no col for input feature `y`'

In [14]:
Candidates(**json.loads(response.content)).to_pandas()

,x_1,x_2,y_pred,y_sd,y_des
0,-0.24894,1.374407,42.664204,249.896859,-42.664204


In [16]:
Candidates(**json.loads(response.content)).to_pandas()

,x_1,x_2
0,3.907609,0.900517
1,-2.052400,4.601854
2,-4.968211,-4.380276
3,-1.602816,-4.190831
4,4.727611,-4.774569
